### Init

In [1]:
import os
import argparse

import torch
import os
import argparse
import matplotlib.pyplot as plt

import torch
import torchvision.transforms as transforms

from os2d.modeling.model import build_os2d_from_config
from os2d.config import cfg
import  os2d.utils.visualization as visualizer
from os2d.structures.feature_map import FeatureMapSize
from os2d.utils import setup_logger, read_image, get_image_size_after_resize_preserving_aspect_ratio
from os2d.data import dataloader
from os2d.modeling.model import build_os2d_from_config

from os2d.data.dataloader import build_eval_dataloaders_from_cfg, build_train_dataloader_from_config
from os2d.engine.train import trainval_loop
from os2d.utils import set_random_seed, get_trainable_parameters, mkdir, save_config, setup_logger, get_data_path
from os2d.engine.optimization import create_optimizer
from os2d.config import cfg
from os2d.utils.visualization import *
import random
import os2d.utils.visualization as visualizer
from pathlib import Path
import cv2
import numpy as np
from os2d.utils import get_image_size_after_resize_preserving_aspect_ratio
from src.util.detection import generate_detection_boxes
from src.util.visualize import visualize_boxes_on_image
from src.util.filter import DataLoaderDB

In [2]:
if cfg.is_cuda:
    assert torch.cuda.is_available(), "Do not have available GPU, but cfg.is_cuda == 1"
    torch.backends.cudnn.benchmark = True

# random seed
set_random_seed(cfg.random_seed, cfg.is_cuda)

# Model
net, box_coder, criterion, img_normalization, optimizer_state = build_os2d_from_config(cfg)

# Optimizer
parameters = get_trainable_parameters(net)
optimizer = create_optimizer(parameters, cfg.train.optim, optimizer_state)

# load the dataset
data_path = get_data_path()
dataloader_train, datasets_train_for_eval = build_train_dataloader_from_config(cfg, box_coder, img_normalization,
                                                                                data_path=data_path)

dataloaders_eval = build_eval_dataloaders_from_cfg(cfg, box_coder, img_normalization,
                                                    datasets_for_eval=datasets_train_for_eval,
                                                    data_path=data_path)

db = DataLoaderDB( path = './src/db/data.csv' , dataloader = dataloader_train)

### Test Basic Method of DB

In [3]:
image_ids = list ( map( int , db.get_image_ids()) )
sorted_image_ids = sorted(image_ids)
print( sorted_image_ids )

[]


In [4]:
db.get_class_ids_by_image_id(0)

{}

In [5]:
len(db.get_value_by_id( 10 , 42 ))

0

### Test Basic Method of ContextAoiAlign

In [6]:
from src.lcp.ct_aoi_align import ContextAoiAlign
transform_image = transforms.Compose([
                      transforms.ToTensor(),
                      transforms.Normalize(img_normalization["mean"], img_normalization["std"])
                      ])

context_aoi_align = ContextAoiAlign( db, dataloader_train, transform_image , net , cfg )

In [7]:
# context_aoi_align.compute_roi_region_for_all()

In [8]:
context_aoi_align.get_feature_map( 0 )

tensor([[[2.6359, 0.7501, 0.5805,  ..., 0.2295, 0.4535, 1.1069],
         [0.6802, 0.3717, 0.0000,  ..., 0.2353, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.2768, 0.5063, 0.0000,  ..., 0.3253, 0.7125, 0.0000],
         [0.3378, 0.0000, 0.0000,  ..., 0.0000, 0.7866, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0829, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.2380, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 2.5919, 3.4586, 0.4234],
         [0.0000, 0.0000, 0.0000,  ..., 0.7022, 1.2554, 0.0000]],

        [[0.0000, 0.6587, 0.0000,  ..., 0.0000, 1.6693, 2.2038],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.3089, 1.7788],
         [0.0000, 0.3178, 0.0000,  ..., 0.0000, 0.9639, 1.

In [9]:
context_aoi_align.get_feature_map( 0 ).shape

torch.Size([1024, 82, 109])

In [10]:
context_aoi_align.get_feature_map_no_grad( 0 )

tensor([[[2.6359, 0.7501, 0.5805,  ..., 0.2295, 0.4535, 1.1069],
         [0.6802, 0.3717, 0.0000,  ..., 0.2353, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.2768, 0.5063, 0.0000,  ..., 0.3253, 0.7125, 0.0000],
         [0.3378, 0.0000, 0.0000,  ..., 0.0000, 0.7866, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0829, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.2380, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 2.5919, 3.4586, 0.4234],
         [0.0000, 0.0000, 0.0000,  ..., 0.7022, 1.2554, 0.0000]],

        [[0.0000, 0.6587, 0.0000,  ..., 0.0000, 1.6693, 2.2038],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.3089, 1.7788],
         [0.0000, 0.3178, 0.0000,  ..., 0.0000, 0.9639, 1.

In [11]:
context_aoi_align.get_feature_map_no_grad( 0 ).shape

torch.Size([1024, 82, 109])

In [12]:
# context_aoi_align.extract_roi_features_by_ids( 0 , 0 )

In [13]:
context_aoi_align.extract_roi_features_by_ids_in_no_grad( 0, 0 )

[]

In [14]:
context_aoi_align.roi_align( 0 , 0 , 
                            (350.8800048828125, 0.0) , (767.0399780273438, 595.598388671875), 
                            (350.8800048828125, 0.0) , (783.6328192660219, 626.5802359988452) 
                            )

{'truth_feature': tensor([[[4.2222e-01, 0.0000e+00, 5.6205e-02,  ..., 7.1005e-01,
           6.0172e-01, 1.6310e+00],
          [4.9514e-02, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           4.1607e-02, 5.1353e-01],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 3.1874e-02],
          ...,
          [1.5927e-01, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00]],
 
         [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          ...,
          [0.0000e+00, 0.0000e+00, 0.0000

In [15]:
context_aoi_align.extract_roi_align_no_grad( 0 , 0 , 
                                            (350.8800048828125, 0.0) , (767.0399780273438, 595.598388671875), 
                                            (350.8800048828125, 0.0) , (783.6328192660219, 626.5802359988452) 
                                            )

{'truth_feature': tensor([[[4.2222e-01, 0.0000e+00, 5.6205e-02,  ..., 7.1005e-01,
           6.0172e-01, 1.6310e+00],
          [4.9514e-02, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           4.1607e-02, 5.1353e-01],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 3.1874e-02],
          ...,
          [1.5927e-01, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00]],
 
         [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          ...,
          [0.0000e+00, 0.0000e+00, 0.0000

In [16]:
# context_aoi_align.extract_roi_features_for_all()

In [17]:
from src.lcp.aux_net import AuxiliaryNetwork
aux_net = AuxiliaryNetwork( context_aoi_align, db )

In [ ]:
point1_x = float( db.get_specific_data(0 , 0 , 'point1_x' )[0] )
point1_y = float( db.get_specific_data(0 , 0 , 'point1_y' )[0] )
point2_x = float( db.get_specific_data(0 , 0 , 'point2_x' )[0] )
point2_y = float( db.get_specific_data(0 , 0 , 'point2_y' )[0] )   

point1 = ( point1_x , point1_y )
point2 = ( point2_x , point2_y )
image_id = 0
class_id = 0 
m = 5

IndexError: list index out of range

In [ ]:
print("=== 測試回歸損失近似函數 ===")
ar_loss = aux_net.approximate_ar_loss_no_grad(image_id, class_id, point1, point2, m = 3)
print(f"Image {image_id}, Class {class_id}: AR Loss = {ar_loss:.4f}")
print(f"返回類型: {type(ar_loss)}")
print(f"數值檢查: {'✓ 正常' if ar_loss >= 0 else '✗ 異常'}")

In [ ]:
print("\n=== 測試分類損失近似函數 ===")
ac_loss = aux_net.approximate_ac_loss_no_grad(image_id, class_id, point1, point2)
print(f"Image {image_id}, Class {class_id}: AC Loss = {ac_loss:.4f}")
print(f"返回類型: {type(ac_loss)}")
print(f"數值檢查: {'✓ 正常' if ac_loss >= 0 else '✗ 異常'}")

In [ ]:
# 測試 Context RoI Align 整合
print("\n=== 測試 Context RoI Align 整合 ===")

# 1. 測試 Context ROI 獲取
context_roi_point1, context_roi_point2 = aux_net._context_roi_align._get_roi_region_specific(
    image_id, class_id, point1, point2
)
print(f"Context ROI: {context_roi_point1} -> {context_roi_point2}")

# 2. 測試特徵提取
contextual_features = aux_net._context_roi_align.extract_roi_align_no_grad(
    image_id, class_id, point1, point2,
    context_roi_point1, context_roi_point2
)

if contextual_features:
    print(f"特徵提取: ✓ 成功")
    combined_feature = contextual_features['combined_feature']
    print(f"特徵維度: {combined_feature.shape}")
    
    import torch
    print(f"特徵統計: mean={torch.mean(combined_feature):.4f}, std={torch.std(combined_feature):.4f}")
else:
    print("特徵提取: ✗ 失敗")


In [ ]:
# 性能測試
import time

print("\n=== 性能測試 ===")

# 測試執行時間
iterations = 5
start_time = time.time()

for i in range(iterations):
    ar_loss = aux_net.approximate_ar_loss_no_grad(image_id, class_id, point1, point2)
    ac_loss = aux_net.approximate_ac_loss_no_grad(image_id, class_id, point1, point2)

end_time = time.time()
avg_time = (end_time - start_time) / iterations

print(f"平均執行時間: {avg_time:.4f} 秒")
print(f"執行 {iterations} 次總時間: {end_time - start_time:.4f} 秒")


In [ ]:
# 詳細調試
print("\n=== 詳細調試執行 ===")

print(f"輸入參數: image_id={image_id}, class_id={class_id}")
print(f"邊界框: {point1} -> {point2}")

# 步驟 1: Context ROI 獲取
print("\n步驟 1: Context ROI 獲取")
context_roi_point1, context_roi_point2 = aux_net._context_roi_align._get_roi_region_specific(
    image_id, class_id, point1, point2
)
print(f"結果: {context_roi_point1} -> {context_roi_point2}")

# 步驟 2: 特徵提取
print("\n步驟 2: 特徵提取")
contextual_features = aux_net._context_roi_align.extract_roi_align_no_grad(
    image_id, class_id, point1, point2,
    context_roi_point1, context_roi_point2
)

if contextual_features:
    combined_feature = contextual_features['combined_feature']
    print(f"特徵維度: {combined_feature.shape}")
    
    import torch
    global_mean = torch.mean(combined_feature)
    global_std = torch.std(combined_feature)
    print(f"全域統計: mean={global_mean:.4f}, std={global_std:.4f}")
    
    # 步驟 3: 統計計算
    print("\n步驟 3: 統計計算")
    C, H, W = combined_feature.shape
    channel_means = torch.mean(combined_feature.view(C, -1), dim=1)
    mean_deviations = torch.abs(channel_means - global_mean)
    localization_sensitivity = torch.mean(mean_deviations) / (global_std + 1e-8)
    print(f"定位敏感性: {localization_sensitivity:.4f}")

# 步驟 4: GIoU 統計
print("\n步驟 4: GIoU 統計")
giou_stats = aux_net._compute_giou_statistics(image_id, class_id)
print(f"GIoU 統計: {giou_stats}")

# 步驟 5: 最終損失計算
print("\n步驟 5: 最終損失計算")
ar_loss = aux_net.approximate_ar_loss_no_grad(image_id, class_id, point1, point2, m = 3)
ac_loss = aux_net.approximate_ac_loss_no_grad(image_id, class_id, point1, point2)
print(f"AR Loss: {ar_loss:.4f}")
print(f"AC Loss: {ac_loss:.4f}")
print(f"Total: {ar_loss + ac_loss:.4f}")


#### Test for LCP

In [ ]:
from src.lcp.lcp import LCP


In [ ]:
lcp = LCP(net, aux_net, dataloader_train)
lcp.init_for_indices()

In [ ]:
img_tensor = lcp.get_image_tensor_from_dataloader(image_id=0)


In [ ]:
layers = lcp.get_layers_name()
for name, ch in layers:
    print(f"{name}: {ch} channels")

In [ ]:
layer_name = 'net_feature_maps.layer3.2.conv2'  # 根據您的網路架構調整
lambda_rate = 1.0
use_image_num = 3  # 使用少量圖像進行測試

print(f"測試層: {layer_name}")
print(f"Lambda 率: {lambda_rate}")
print(f"使用圖像數: {use_image_num}")

In [ ]:
# 檢查設備一致性
print("=== 設備一致性檢查 ===")

# 檢查網路設備
net_device = next(lcp._net.parameters()).device
prune_net_device = next(lcp._prune_net.parameters()).device

print(f"主網路設備: {net_device}")
print(f"剪枝網路設備: {prune_net_device}")

# 檢查圖像張量設備
img_tensor = lcp.get_image_tensor_from_dataloader(0, is_cuda=False)
print(f"圖像張量設備: {img_tensor.device}")


In [ ]:
# # 執行無梯度通道重要性計算
# print("\n=== 執行無梯度通道重要性計算 ===")

# try:
#     importance_scores = lcp.compute_channel_importance_no_grad(
#         layer_name=layer_name,
#         lambda_rate=lambda_rate,
#         use_image_num=use_image_num,
#         random_seed=42
#     )
    
#     print(f"✅ 計算成功！")
#     print(f"通道數量: {len(importance_scores)}")
#     print(f"重要性分數範圍: {np.min(importance_scores):.4f} - {np.max(importance_scores):.4f}")
#     print(f"平均重要性: {np.mean(importance_scores):.4f}")
#     print(f"標準差: {np.std(importance_scores):.4f}")
    
#     # 顯示前10個通道的重要性
#     print(f"\n前10個通道重要性分數:")
#     for i in range(min(10, len(importance_scores))):
#         print(f"  通道 {i}: {importance_scores[i]:.4f}")
        
# except Exception as e:
#     print(f"❌ 計算失敗: {e}")
#     import traceback
#     traceback.print_exc()


In [ ]:
# # 測試各個組件功能
# print("\n=== 詳細組件測試 ===")

# # 測試圖像數據準備
# image_id_all = list(map(int, lcp._get_db().get_image_ids()))
# print(f"總圖像數: {len(image_id_all)}")
# print(f"前5個圖像ID: {image_id_all[:5]}")

# # 測試單個圖像的特徵統計計算
# test_image_id = image_id_all[0]
# print(f"\n測試圖像 {test_image_id} 的特徵統計:")

# try:
#     feature_stats = lcp._extract_and_compute_theoretical_statistics(test_image_id, layer_name)
    
#     if feature_stats:
#         print(f"✅ 特徵統計計算成功")
#         print(f"通道數: {len(feature_stats['channels'])}")
#         print(f"全域統計: mean={feature_stats['global_mean']:.4f}, std={feature_stats['global_std']:.4f}")
        
#         # 顯示前3個通道的詳細統計
#         print(f"\n前3個通道詳細統計:")
#         for i in range(min(3, len(feature_stats['channels']))):
#             ch_stat = feature_stats['channels'][i]
#             print(f"  通道 {i}:")
#             print(f"    L1範數: {ch_stat['l1_norm']:.4f}")
#             print(f"    方差: {ch_stat['variance']:.4f}")
#             print(f"    均值偏差: {ch_stat['mean_deviation']:.4f}")
#             print(f"    能量: {ch_stat['energy']:.4f}")
#             print(f"    稀疏性: {ch_stat['sparsity']:.4f}")
#             print(f"    重要性: {ch_stat['importance']:.4f}")
#     else:
#         print("❌ 特徵統計計算失敗")
        
# except Exception as e:
#     print(f"❌ 特徵統計測試失敗: {e}")


In [ ]:
# # 驗證數學公式實現
# print("\n=== 數學公式驗證 ===")

# if feature_stats:
#     # 手動驗證統合公式
#     test_channel = feature_stats['channels'][0]
#     global_std = feature_stats['global_std']
    
#     # 手動計算重要性分數
#     manual_importance = (
#         0.3 * (test_channel['l1_norm'] / global_std) +                    # w1×L1/σ
#         0.2 * (test_channel['variance'] / (global_std ** 2)) +            # w2×Var/σ²
#         0.2 * (test_channel['mean_deviation'] / global_std) +             # w3×|μ-μ_g|/σ
#         0.15 * (test_channel['energy'] / (global_std ** 2)) +             # w4×E/σ²
#         0.15 * (1.0 - test_channel['sparsity'])                           # w5×(1-sparsity)
#     )
    
#     computed_importance = test_channel['importance']
    
#     print(f"手動計算重要性: {manual_importance:.6f}")
#     print(f"函數計算重要性: {computed_importance:.6f}")
#     print(f"差異: {abs(manual_importance - computed_importance):.8f}")
#     print(f"公式驗證: {'✅ 正確' if abs(manual_importance - computed_importance) < 1e-6 else '❌ 錯誤'}")


In [ ]:
# # 性能測試
# print("\n=== 性能測試 ===")

# import time
# import psutil
# import os

# process = psutil.Process(os.getpid())

# # 測試執行時間和記憶體使用
# start_memory = process.memory_info().rss / 1024 / 1024  # MB
# start_time = time.time()

# # 執行多次測試
# test_iterations = 3
# for i in range(test_iterations):
#     print(f"執行第 {i+1} 次測試...")
#     importance_scores = lcp.compute_channel_importance_no_grad(
#         layer_name=layer_name,
#         lambda_rate=lambda_rate,
#         use_image_num=2,  # 減少圖像數以加快測試
#         random_seed=42
#     )

# end_time = time.time()
# end_memory = process.memory_info().rss / 1024 / 1024  # MB

# avg_time = (end_time - start_time) / test_iterations
# memory_usage = end_memory - start_memory

# print(f"\n性能統計:")
# print(f"  平均執行時間: {avg_time:.4f} 秒")
# print(f"  記憶體使用增量: {memory_usage:.2f} MB")
# print(f"  總執行時間: {end_time - start_time:.4f} 秒")


In [ ]:
# # 通道重要性排序測試
# print("\n=== 通道排序測試 ===")

# if 'importance_scores' in locals():
#     # 獲取重要性排序
#     sorted_indices = np.argsort(importance_scores)[::-1]  # 降序排列
    
#     print(f"最重要的前10個通道:")
#     for i in range(min(10, len(sorted_indices))):
#         channel_idx = sorted_indices[i]
#         importance = importance_scores[channel_idx]
#         print(f"  排名 {i+1}: 通道 {channel_idx} (重要性: {importance:.4f})")
    
#     print(f"\n最不重要的後5個通道:")
#     for i in range(max(0, len(sorted_indices)-5), len(sorted_indices)):
#         channel_idx = sorted_indices[i]
#         importance = importance_scores[channel_idx]
#         rank = len(sorted_indices) - i
#         print(f"  倒數第 {rank}: 通道 {channel_idx} (重要性: {importance:.4f})")
    
#     # 統計分析
#     top_10_percent = int(len(importance_scores) * 0.1)
#     top_scores = importance_scores[sorted_indices[:top_10_percent]]
#     bottom_scores = importance_scores[sorted_indices[-top_10_percent:]]
    
#     print(f"\n統計分析:")
#     print(f"  前10%通道平均重要性: {np.mean(top_scores):.4f}")
#     print(f"  後10%通道平均重要性: {np.mean(bottom_scores):.4f}")
#     print(f"  重要性比率: {np.mean(top_scores) / np.mean(bottom_scores):.2f}")


In [ ]:
# # 測試總結
# print("\n" + "="*60)
# print("🎯 LCP 無梯度通道重要性計算測試總結")
# print("="*60)

# test_results = {
#     '基礎計算': 'importance_scores' in locals() and len(importance_scores) > 0,
#     '特徵統計': 'feature_stats' in locals() and feature_stats is not None,
#     '數學公式': 'manual_importance' in locals(),
#     '性能測試': 'avg_time' in locals(),
#     '排序分析': 'sorted_indices' in locals()
# }

# print("測試項目結果:")
# for test_name, result in test_results.items():
#     status = "✅ 通過" if result else "❌ 失敗"
#     print(f"  {test_name}: {status}")

# overall_success = all(test_results.values())
# print(f"\n🏆 整體測試結果: {'🎉 全部通過！' if overall_success else '⚠️ 部分項目需要檢查'}")

# if overall_success:
#     print("\n✨ 您的 LCP 無梯度通道重要性計算實現完全正確！")
#     print("📊 統合數學公式正確實現")
#     print("🚀 無梯度架構完美整合")
#     print("🎯 準備好進行實際的通道剪枝應用！")
# else:
#     print("\n🔧 請檢查失敗的測試項目並進行相應調整。")


In [ ]:
# # 例：剪掉 layer3.2.conv2 一半通道
# keep, discard = lcp.get_channel_selection_by_no_grad(
#     layer_name   = "net_feature_maps.layer3.2.conv2",
#     discard_rate = 0.5,
#     lambda_rate  = 1.0,
#     use_image_num= 5,
#     random_seed  = 42
# )

# print("保留通道:", keep)
# print("捨棄通道:", discard)


### Create DB for keep indices

In [ ]:
from src.util.prune_db import PruneDBControler
prune_db = PruneDBControler( path = './src/db/prune_channel_information.csv' )
prune_db.initial()

In [ ]:

layers = lcp.get_layers_name()
for name, ch in layers:
    if name == 'layer2.0.conv2' or name == 'layer3.0.conv2':
        pass
    else:
        continue
    print(f"{name}: {ch} channels")
    keep, discard = lcp.get_channel_selection_by_no_grad(
        layer_name   = f"net_feature_maps.{name}",
        discard_rate = 0.5,
        lambda_rate  = 1.0,
        use_image_num= 3,
        random_seed  = 42
    )
    print(f"layer {name} , 預計保留通道數量: {len(keep)}/{ch}, 預計捨棄通道數量: {len(discard)}/{ch}")
    prune_db.write_data(
        layer = f"net_feature_maps.{name}",
        original_channel_num= len(keep) + len(discard),
        num_of_keep_channel = len(keep),
        keep_index  = keep
    )

### Test for Pruner

In [ ]:
from src.lcp.pruner import Pruner
pruner = Pruner( lcp._prune_net )
pruner.set_prune_db( prune_db )

In [ ]:
# dependencies = pruner.resolve_layer_dependencies('net_feature_maps.layer2.0.conv3')
# print("依賴關係分析結果:")
# print(f"目標層級: {dependencies['target']}")
# print(f"BatchNorm 層: {dependencies['batch_norm']}")
# print(f"下游層級: {dependencies['downstream_layers']}")
# print(f"跳躍連接: {dependencies['skip_connections']}")
# print(f"剪枝策略: {dependencies['pruning_strategy']}")

In [ ]:
# dependencies = pruner.resolve_layer_dependencies('net_feature_maps.layer2.0.conv3')
# print("依賴關係分析結果:")
# print(f"目標層級: {dependencies['target']}")
# print(f"BatchNorm 層: {dependencies['batch_norm']}")
# print(f"下游層級: {dependencies['downstream_layers']}")
# print(f"跳躍連接: {dependencies['skip_connections']}")
# print(f"剪枝策略: {dependencies['pruning_strategy']}")

In [ ]:
# for name, ch in layers:
#     dependencies = pruner.resolve_layer_dependencies(f'net_feature_maps.{name}')
#     print("依賴關係分析結果:")
#     print(f"目標層級: {dependencies['target']}")
#     print(f"BatchNorm 層: {dependencies['batch_norm']}")
#     print(f"下游層級: {dependencies['downstream_layers']}")
#     print(f"跳躍連接: {dependencies['skip_connections']}")
#     print(f"剪枝策略: {dependencies['pruning_strategy']}")

In [ ]:
# # 測試改進後的依賴分析
# layer_name = "net_feature_maps.layer3.0.conv3"
# dependencies = pruner.resolve_layer_dependencies(layer_name)

# print("依賴分析結果:")
# print(f"BatchNorm 層: {dependencies['batch_norm']}")
# print(f"下游層級: {dependencies['downstream_layers']}")
# print(f"Downsample 層: {dependencies['downsample_layer']}")
# print(f"跳躍連接: {dependencies['skip_connections']}")
# print(f"剪枝策略: {dependencies['pruning_strategy']}")

# # 執行剪枝
# if dependencies['downsample_layer']:
#     print(f"✅ 找到 downsample 層: {dependencies['downsample_layer']}")
# else:
#     print("ℹ️  該層沒有對應的 downsample 層")


In [ ]:
# # 測試參數設定
# layer_name = 'net_feature_maps.layer2.0.conv2'
# keep_indexes = prune_db.get_layer_keep_indices(layer_name)
# dependencies = pruner.resolve_layer_dependencies(layer_name)
# print(f"🔍 測試層級: {layer_name}")
# print(f"📊 保留通道索引: {keep_indexes}")
# print(f"📈 保留通道數: {len(keep_indexes)}")


In [ ]:
# print( dependencies )

In [ ]:
# print( pruner.prune_network)

In [ ]:
# # === 步驟 1: 輸出通道剪枝 ===
# print("\n=== 步驟 1: 輸出通道剪枝 ===")
# success = pruner._prune_out_channel(layer_name, keep_indexes)

# if success:
#     print("✅ 輸出通道剪枝成功")
    
#     # 驗證輸出通道剪枝結果
#     target_layer = pruner._get_layer_by_name(layer_name)
#     print(f"   剪枝後輸出通道數: {target_layer.out_channels}")
#     print(f"   權重形狀: {target_layer.weight.shape}")
# else:
#     print("❌ 輸出通道剪枝失敗")

# # === 步驟 2: BatchNorm 剪枝 ===
# print("\n=== 步驟 2: BatchNorm 剪枝 ===")
# bn_layer_name = dependencies.get('batch_norm')
# print(f"🎯 BatchNorm 層: {bn_layer_name}")

# if bn_layer_name:
#     success = pruner._prune_batchnorm_layer(bn_layer_name, keep_indexes)
    
#     if success:
#         print("✅ BatchNorm 剪枝成功")
        
#         # 驗證 BatchNorm 剪枝結果
#         bn_layer = pruner._get_layer_by_name(bn_layer_name)
#         print(f"   BatchNorm 特徵數: {bn_layer.num_features}")
#         print(f"   權重形狀: {bn_layer.weight.shape}")
#         print(f"   偏置形狀: {bn_layer.bias.shape}")
#     else:
#         print("❌ BatchNorm 剪枝失敗")
# else:
#     print("⚠️  未找到對應的 BatchNorm 層")

# # === 步驟 3: 下游層輸入通道剪枝 ===
# print("\n=== 步驟 3: 下游層輸入通道剪枝 ===")
# downstream_layers = dependencies.get('downstream_layers', [])

# if downstream_layers:
#     next_layer_name = downstream_layers[0]
#     print(f"🎯 下游層: {next_layer_name}")

#     # 檢查下游層剪枝前的狀態
#     next_layer = pruner._get_layer_by_name(next_layer_name)
#     if next_layer:
#         print(f"剪枝前下游層輸入通道數: {next_layer.in_channels}")
#         print(f"剪枝前下游層權重形狀: {next_layer.weight.shape}")

#         # 執行下游層輸入通道剪枝
#         success = pruner._prune_next_layer_inputs(layer_name, next_layer_name, keep_indexes)

#         if success:
#             print("✅ 下游層輸入通道剪枝成功")
            
#             # 驗證下游層剪枝結果
#             next_layer = pruner._get_layer_by_name(next_layer_name)
#             print(f"   剪枝後輸入通道數: {next_layer.in_channels}")
#             print(f"   剪枝後權重形狀: {next_layer.weight.shape}")
            
#             # 驗證維度一致性
#             if next_layer.in_channels == len(keep_indexes):
#                 print("   ✅ 維度一致性檢查通過")
#             else:
#                 print("   ❌ 維度一致性檢查失敗")
#         else:
#             print("❌ 下游層輸入通道剪枝失敗")
#     else:
#         print("❌ 無法獲取下游層對象")
# else:
#     print("⚠️  未找到下游層")

# # === 步驟 4: Downsample 剪枝 ===
# print("\n=== 步驟 4: Downsample 剪枝 ===")
# skip_connections = dependencies.get('skip_connections', [])

# if skip_connections:
#     print(f"🎯 跳躍連接: {skip_connections}")
    
#     # 檢查是否需要處理 downsample
#     downsample_processed = False
    
#     for skip_conn in skip_connections:
#         if 'downsample' in skip_conn:
#             print(f"🔧 處理 Downsample 層: {skip_conn}")
            
#             # 檢查 downsample 層是否存在
#             if pruner._verify_layer_exists(skip_conn):
#                 downsample_layer = pruner._get_layer_by_name(skip_conn)
                
#                 if downsample_layer:
#                     print(f"   Downsample 層類型: {type(downsample_layer).__name__}")
                    
#                     # 檢查 downsample 剪枝前的狀態
#                     if hasattr(downsample_layer, '__iter__'):  # Sequential
#                         print("   Downsample 結構分析:")
#                         for i, sub_layer in enumerate(downsample_layer):
#                             print(f"     [{i}] {type(sub_layer).__name__}", end="")
#                             if hasattr(sub_layer, 'out_channels'):
#                                 print(f" - 輸出通道: {sub_layer.out_channels}")
#                             elif hasattr(sub_layer, 'num_features'):
#                                 print(f" - 特徵數: {sub_layer.num_features}")
#                             else:
#                                 print()
                    
#                     # 執行 downsample 剪枝
#                     try:
#                         pruner._prune_downsample_connection(skip_conn, keep_indexes)
#                         print("   ✅ Downsample 剪枝成功")
#                         downsample_processed = True
                        
#                         # 驗證 downsample 剪枝結果
#                         downsample_layer = pruner._get_layer_by_name(skip_conn)
#                         if hasattr(downsample_layer, '__iter__'):
#                             print("   剪枝後 Downsample 結構:")
#                             for i, sub_layer in enumerate(downsample_layer):
#                                 print(f"     [{i}] {type(sub_layer).__name__}", end="")
#                                 if hasattr(sub_layer, 'out_channels'):
#                                     print(f" - 輸出通道: {sub_layer.out_channels}")
#                                 elif hasattr(sub_layer, 'num_features'):
#                                     print(f" - 特徵數: {sub_layer.num_features}")
#                                 else:
#                                     print()
                        
#                         # 驗證維度一致性
#                         conv_layer = downsample_layer[0] if hasattr(downsample_layer, '__iter__') else downsample_layer
#                         if hasattr(conv_layer, 'out_channels') and conv_layer.out_channels == len(keep_indexes):
#                             print("   ✅ Downsample 維度一致性檢查通過")
#                         else:
#                             print("   ❌ Downsample 維度一致性檢查失敗")
#                     except Exception as e:
#                         print(f"   ❌ Downsample 剪枝失敗: {e}")
#                 else:
#                     print(f"   ❌ 無法獲取 Downsample 層對象: {skip_conn}")
#             else:
#                 print(f"   ⚠️  Downsample 層不存在: {skip_conn}")
        
#         elif skip_conn == 'residual_addition':
#             print("   🔗 檢測到殘差連接，但無需額外處理")
    
#     if not downsample_processed:
#         print("   ℹ️  無需處理 Downsample 層")
# else:
#     print("⚠️  未檢測到跳躍連接")

# # === 步驟 5: 整體驗證 ===
# print("\n=== 步驟 5: 整體驗證 ===")

# # 驗證剪枝一致性
# print("🔍 剪枝一致性檢查:")

# # 1. 檢查目標層
# target_layer = pruner._get_layer_by_name(layer_name)
# if target_layer:
#     print(f"   目標層 {layer_name}: {target_layer.out_channels} 通道")

# # 2. 檢查 BatchNorm 層
# if bn_layer_name:
#     bn_layer = pruner._get_layer_by_name(bn_layer_name)
#     if bn_layer:
#         print(f"   BatchNorm {bn_layer_name}: {bn_layer.num_features} 特徵")
#         if bn_layer.num_features == target_layer.out_channels:
#             print("   ✅ 目標層與 BatchNorm 維度一致")
#         else:
#             print("   ❌ 目標層與 BatchNorm 維度不一致")

# # 3. 檢查下游層
# if downstream_layers:
#     next_layer = pruner._get_layer_by_name(downstream_layers[0])
#     if next_layer:
#         print(f"   下游層 {downstream_layers[0]}: {next_layer.in_channels} 輸入通道")
#         if next_layer.in_channels == target_layer.out_channels:
#             print("   ✅ 目標層與下游層維度一致")
#         else:
#             print("   ❌ 目標層與下游層維度不一致")

# # 4. 檢查 Downsample 層
# downsample_layers = [conn for conn in skip_connections if 'downsample' in conn]
# if downsample_layers:
#     for downsample_name in downsample_layers:
#         if pruner._verify_layer_exists(downsample_name):
#             downsample_layer = pruner._get_layer_by_name(downsample_name)
#             if downsample_layer and hasattr(downsample_layer, '__iter__'):
#                 conv_layer = downsample_layer[0]
#                 if hasattr(conv_layer, 'out_channels'):
#                     print(f"   Downsample {downsample_name}: {conv_layer.out_channels} 輸出通道")
#                     if conv_layer.out_channels == target_layer.out_channels:
#                         print("   ✅ 目標層與 Downsample 維度一致")
#                     else:
#                         print("   ❌ 目標層與 Downsample 維度不一致")


In [ ]:
# # === 步驟 7: 通道索引追蹤測試 ===
# print("\n=== 步驟 7: 通道索引追蹤測試 ===")

# print("🎯 執行通道索引追蹤...")

# try:
#     # 執行通道索引追蹤
#     pruner.track_channel_indices(layer_name, keep_indexes)
#     print("   ✅ 通道索引追蹤成功")
    
#     # 驗證追蹤結果
#     print("   🔍 追蹤結果驗證:")
    
#     # 1. 檢查剪枝數據庫是否更新
#     if hasattr(pruner, 'prune_db') and pruner.prune_db:
#         try:
#             stored_indices = pruner.prune_db.get_layer_keep_indices(layer_name)
#             if stored_indices:
#                 print(f"     - 數據庫中保存的索引數量: {len(stored_indices)}")
#                 print(f"     - 索引一致性檢查: {'✅ 一致' if stored_indices == keep_indexes else '❌ 不一致'}")
#             else:
#                 print("     - ⚠️  數據庫中未找到保存的索引")
#         except Exception as e:
#             print(f"     - ❌ 數據庫檢查失敗: {e}")
    
#     # 2. 檢查內部追蹤記錄
#     if hasattr(pruner, '_prune_history'):
#         if layer_name in pruner._prune_history:
#             history = pruner._prune_history[layer_name]
#             print(f"     - 原始通道數: {history.get('original_channels', 'N/A')}")
#             print(f"     - 保留通道數: {history.get('kept_channels', 'N/A')}")
#             print(f"     - 剪枝比例: {history.get('prune_ratio', 0):.2%}")
#         else:
#             print("     - ⚠️  內部追蹤記錄中未找到該層")
    
#     # 3. 計算剪枝統計
#     original_channels = target_layer.out_channels if target_layer else len(keep_indexes)
#     kept_channels = len(keep_indexes)
#     prune_ratio = 1.0 - (kept_channels / original_channels) if original_channels > 0 else 0.0
    
#     print(f"     - 實際剪枝統計:")
#     print(f"       * 保留通道: {kept_channels}")
#     print(f"       * 剪枝比例: {prune_ratio:.2%}")
#     print(f"       * 壓縮效果: {(1-prune_ratio)*100:.1f}% 保留")
    
# except Exception as e:
#     print(f"   ❌ 通道索引追蹤失敗: {e}")
#     import traceback
#     traceback.print_exc()


In [ ]:
# # === 步驟 8: 完整性驗證與測試報告 ===
# print("\n=== 步驟 8: 完整性驗證與測試報告 ===")

# # 網路前向傳播測試
# print("🧪 網路前向傳播測試:")
# try:
#     # 創建測試輸入
#     device = next(pruner.prune_network.parameters()).device
#     test_input = torch.randn(1, 3, 224, 224, device=device)
    
#     # 設置網路為評估模式
#     pruner.prune_network.eval()
    
#     with torch.no_grad():
#         # 嘗試前向傳播
#         output = pruner.prune_network.net_feature_maps(test_input)
#         print(f"   ✅ 前向傳播測試成功")
#         print(f"   📊 輸入形狀: {test_input.shape}")
#         print(f"   📊 輸出形狀: {output.shape}")
        
# except Exception as e:
#     print(f"   ❌ 前向傳播測試失敗: {e}")

# # 生成測試報告
# print("\n📋 === 完整測試報告 ===")

# test_results = {
#     '輸出通道剪枝': True,  # 根據實際結果設置
#     'BatchNorm 剪枝': True,
#     '下游層輸入剪枝': True,
#     'Downsample 剪枝': downsample_processed if 'downsample_processed' in locals() else False,
#     '輸入通道修正': True,  # 根據實際結果設置
#     '通道索引追蹤': True,
#     '前向傳播測試': True   # 根據實際結果設置
# }

# print(f"🎯 測試層級: {layer_name}")
# print(f"📊 保留通道數: {len(keep_indexes)}")
# print(f"📈 剪枝比例: {(1 - len(keep_indexes) / 512) * 100:.1f}%")  # 假設原始通道數為 512

# print("\n🔍 測試結果詳情:")
# for test_name, result in test_results.items():
#     status = "✅ 通過" if result else "❌ 失敗"
#     print(f"  {test_name}: {status}")

# overall_success = all(test_results.values())
# print(f"\n🏆 整體測試結果: {'🎉 全部通過！' if overall_success else '⚠️ 部分測試失敗'}")

# if overall_success:
#     print("\n✨ 恭喜！您的 LCP 通道剪枝功能運作正常")
#     print("🚀 包含 Track 和 Fix 功能的完整剪枝流程已驗證")
#     print("📝 建議進行更多層級的測試以確保穩定性")
# else:
#     failed_tests = [name for name, result in test_results.items() if not result]
#     print(f"\n🔧 需要檢查的項目: {', '.join(failed_tests)}")
#     print("💡 建議逐項檢查失敗的功能並進行修正")

# print(f"\n🎉 剪枝流程完成！保留 {len(keep_indexes)} 個通道")


In [ ]:
# pruner.print_detailed_layers()

In [ ]:
# print( pruner.prune_network )

In [ ]:
# # === 維度一致性完整驗證 ===
# print("\n=== 維度一致性完整驗證 ===")

# def verify_channel_consistency(pruner, current_layer, next_layer, keep_indexes):
#     """驗證通道維度一致性"""
#     try:
#         current = pruner._get_layer_by_name(current_layer)
#         next_conv = pruner._get_layer_by_name(next_layer)
        
#         print(f"當前層 {current_layer}:")
#         print(f"  輸出通道數: {current.out_channels}")
#         print(f"  保留通道數: {len(keep_indexes)}")
        
#         print(f"下游層 {next_layer}:")
#         print(f"  輸入通道數: {next_conv.in_channels}")
        
#         # 檢查一致性
#         if current.out_channels == len(keep_indexes) == next_conv.in_channels:
#             print("✅ 通道維度完全一致")
#             return True
#         else:
#             print("❌ 通道維度不一致")
#             return False
            
#     except Exception as e:
#         print(f"❌ 驗證過程發生錯誤: {e}")
#         return False

# # 執行驗證
# consistency_ok = verify_channel_consistency(
#     pruner, layer_name, next_layer_name, keep_indexes
# )


##### Test For Prune One Layer 

In [ ]:
# # 測試參數設定
# layer_name = 'net_feature_maps.layer3.0.conv2'
# keep_indexes = prune_db.get_layer_keep_indices(layer_name)
# dependencies = pruner.resolve_layer_dependencies(layer_name)
# print(f"🔍 測試層級: {layer_name}")
# print(f"📊 保留通道索引: {keep_indexes}")
# print(f"📈 保留通道數: {len(keep_indexes)}")


In [ ]:
# pruner.prune_layer(    
#     layer_name   = layer_name
# )

In [ ]:
# print( pruner.prune_network )

In [ ]:

# layers = lcp.get_layers_name()
# for name, ch in layers:
#     if name == 'layer3.0.conv3':
#         pass
#     else:
#         continue
#     print(f"{name}: {ch} channels")
#     keep, discard = lcp.get_channel_selection_by_no_grad(
#         layer_name   = f"net_feature_maps.{name}",
#         discard_rate = 0.5,
#         lambda_rate  = 1.0,
#         use_image_num= 3,
#         random_seed  = 42
#     )
#     print(f"layer {name} , 預計保留通道數量: {len(keep)}/{ch}, 預計捨棄通道數量: {len(discard)}/{ch}")
#     prune_db.write_data(
#         layer = f"net_feature_maps.{name}",
#         original_channel_num= ch,
#         num_of_keep_channel = len(keep),
#         keep_index  = keep
#     )

### Test For Prune Layer in LCP

In [ ]:
print( layers )
layer_names = []
for name, ch in layers:
    if name.endswith('.conv1') or name.endswith('.conv2'):
        layer_names.append( name )

In [ ]:
lcp.set_prune_db(prune_db)

#### Before Prune

In [ ]:
lcp.debug_for_test_vision(
    dataloader_train = dataloader_train,
    img_normalization = img_normalization,
    box_coder = box_coder,
    cfg = cfg,
    count = 2
)

### 這裡要跑 62 minutes for ALL

In [ ]:
for idx, layer_name in enumerate(layer_names):
    if idx != 15:
        continue
    lcp.prune_layer(
        layer_name   = layer_name,
        discard_rate = 0.8,
    )
    break

In [ ]:
print( lcp._prune_net )

#### Test for debug for vision after prune

In [ ]:
lcp.debug_for_test_vision(
    dataloader_train = dataloader_train,
    img_normalization = img_normalization,
    box_coder = box_coder,
    cfg = cfg,
    count = 1
)

In [ ]:
import torch

def count_parameters(model):
    """
    計算模型的參數數量
    Returns:
      total_params: 包含所有參數
      trainable_params: 只包含 requires_grad=True 的參數
    """
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params

import io

def estimate_model_size(model):
    """
    將模型序列化到緩衝區，估算存檔大小（MB）
    """
    buffer = io.BytesIO()
    torch.save(model.state_dict(), buffer)
    size_mb = buffer.getbuffer().nbytes / (1024 ** 2)
    return size_mb



In [ ]:
# 使用範例
print( "原始網路參數統計:\n" )
model = lcp._net  # 已剪枝的網路
total, trainable = count_parameters(model)
print(f"總參數量: {total:,}")
print(f"可訓練參數量: {trainable:,}")

size_mb = estimate_model_size(model)
print(f"模型存儲大小: {size_mb:.2f} MB")

print( "剪枝網路參數統計:\n" )
model = lcp._prune_net  # 已剪枝的網路
total, trainable = count_parameters(model)
print(f"總參數量: {total:,}")
print(f"可訓練參數量: {trainable:,}")

size_mb = estimate_model_size(model)
print(f"模型存儲大小: {size_mb:.2f} MB")

### Test for finetune

In [ ]:
from src.lcp.lcpfinetune import LCPFineTune
lcp_finetune = LCPFineTune(
    prune_net = lcp._prune_net,
    dataloader_train = dataloader_train,
    img_normalization = img_normalization,
    box_coder = box_coder,
    cfg       = cfg,
    optimizer=optimizer,
    parameters=parameters
)

In [ ]:
lcp_finetune._setup_logging()

In [ ]:
cfg.defrost()

cfg.train.optim.max_iter = 10
cfg.train.do_training = True
cfg.output.print_iter = 2
cfg.eval.iter = 5
cfg.train.batch_size = 1
cfg.train.batch_size = 1  # 最小批次
cfg.train.accumulate_grad_batches = 1

# 關閉不必要的功能
if hasattr(cfg.train, 'mining'):
    cfg.train.mining.do_mining = False
if hasattr(cfg.output, 'best_model'):
    cfg.output.best_model.do_get_best_model = False

cfg.freeze()
        

In [ ]:
from src.util.loss import LCPFinetuneCriterion
lcp_criterion = LCPFinetuneCriterion(
    original_criterion=criterion,  # 原始損失函數實例
    aux_net=lcp._aux_net,  # 包含 aux_loss 方法的實例
    auxiliary_weight=1.0  # 可以調整輔助損失權重
)

In [ ]:
lcp_finetune.start_finetune(
    criterion= lcp_criterion,  # 使用自定義損失函數
)

In [ ]:
# lcp_finetune.finetune_one_batch(
#     batch_data = data,
#     lcp_instance = lcp,
#     criterion = criterion
# )

In [ ]:
# lcp_finetune.finetune_loop(
#     lcp_instance = lcp,
#     criterion = criterion,
#     dataloaders_eval = dataloaders_eval,
#     max_iters = 5,
#     print_interval=2
# )